# Chains

In [1]:
import os
import warnings
from dotenv import load_dotenv
from pydantic import BaseModel, BaseConfig
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate

warnings.filterwarnings("ignore")

In [2]:
load_dotenv(".env")
api_key = os.environ.get("key")

In [3]:
chat = ChatOpenAI(api_key=api_key, model='gpt-3.5-turbo', max_tokens=500)

In [4]:
human_template = "{question}"
humna_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([human_template])

In [5]:
chain = LLMChain(llm=chat, prompt=chat_prompt)

In [6]:
chain.run("What is bit quantization")

'Bit quantization is a process of reducing the number of bits used to represent a signal or data. It involves the conversion of continuous analog signals or high-resolution digital signals into a discrete set of values with a limited number of bits. This process is commonly used in digital signal processing and data compression techniques to reduce the storage space or transmission bandwidth required for the signal or data. Bit quantization can result in a loss of detail and accuracy in the representation of the original signal, as the quantization levels are limited by the number of bits used.'

# Simple Sequential Chains

```bash
          --------------------------------------
Input -> |  LLMChain  |  LLMchain  |  LLMchain  | -> Output
          --------------------------------------
```

In [7]:
from langchain.chains import SimpleSequentialChain

In [8]:
template1 = "Give me a simple bullet point outline for a blog post on {topic}"
first_prompt = ChatPromptTemplate.from_template(template1)
chain_one = LLMChain(llm=chat, prompt=first_prompt)

In [9]:
template2 = "Write a blog post tusing this outline {outline}"
second_prompt = ChatPromptTemplate.from_template(template2)
chain_two = LLMChain(llm = chat, prompt=second_prompt)

In [10]:
full_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [11]:
full_chain.run('Skew Data')



> Entering new SimpleSequentialChain chain...
- Introduction to skew data and its significance in data analysis
- Definition of skewness and how it is calculated
- Understanding the different types of skewness: positive skewness, negative skewness, and zero skewness
- Examples of real-life data distributions exhibiting skewness
- Exploring the impact of skewness on statistical measures such as mean, median, and mode
- Techniques for detecting skewness in data through graphical methods (histograms, box plots, etc.)
- Discussing the implications of skewness on statistical analysis and interpretation
- Strategies for handling skewness, including data transformation techniques (logarithmic, square root, etc.)
- Highlighting the importance of addressing skewness before performing certain statistical tests
- Conclusion emphasizing the significance of recognizing and dealing with skewness in data analysis.
Introduction to Skew Data and its Significance in Data Analysis

Data analysis is a c

"Introduction to Skew Data and its Significance in Data Analysis\n\nData analysis is a crucial component of any research or decision-making process. It helps us derive meaningful insights and make informed decisions. However, not all data is created equal. Some datasets exhibit a particular pattern known as skewness, which can significantly impact the analysis and interpretation of the data. In this blog post, we will explore the concept of skewness, its calculation, types, impact on statistical measures, detection techniques, implications on analysis, strategies for handling skewness, and the importance of recognizing and dealing with skewness in data analysis.\n\nDefinition of Skewness and How it is Calculated\n\nSkewness refers to the measure of the asymmetry or lack of symmetry in a dataset's distribution. A symmetrical distribution has equal amounts of data on both sides of the mean, while a skewed distribution has more data concentrated on one side. Skewness can be positive, nega

# Sequential Chains

In [12]:
from langchain.chains import SequentialChain

In [13]:
template1 = "Give a summary of this employee's performance review:\n{review}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain_1 = LLMChain(llm=chat,
                     prompt=prompt1,
                     output_key="review_summary")

In [14]:
template2 = "Identify key employee weaknesses in this review summary:\n{review_summary}"
prompt2 = ChatPromptTemplate.from_template(template2)
chain_2 = LLMChain(llm=chat,
                     prompt=prompt2,
                     output_key="weaknesses")

In [15]:
template3 = "Create a personalized plan to help address and fix these weaknesses:\n{weaknesses}"
prompt3 = ChatPromptTemplate.from_template(template3)
chain_3 = LLMChain(llm=chat,
                     prompt=prompt3,
                     output_key="final_plan")

In [16]:
seq_chain = SequentialChain(chains=[chain_1,chain_2,chain_3],
                            input_variables=['review'],
                            output_variables=['review_summary','weaknesses','final_plan'],
                            verbose=True)

In [17]:
employee_review = '''
Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexibility in handling changing project requirements and learning new technologies. This adaptability allows him to seamlessly transition between different projects and tasks, making him a valuable asset to the team.

Joe's problem-solving skills are exceptional. He approaches issues with a logical mindset and consistently finds effective solutions, often thinking outside the box. His ability to break down complex problems into manageable parts is key to his success in resolving issues efficiently.

Weaknesses:
While Joe possesses numerous strengths, there are a few areas where he could benefit from improvement. One such area is time management. Occasionally, Joe struggles with effectively managing his time, resulting in missed deadlines or the need for additional support to complete tasks on time. Developing better prioritization and time management techniques would greatly enhance his efficiency.

Another area for improvement is Joe's written communication skills. While he communicates well verbally, there have been instances where his written documentation lacked clarity, leading to confusion among team members. Focusing on enhancing his written communication abilities will help him effectively convey ideas and instructions.

Additionally, Joe tends to take on too many responsibilities and hesitates to delegate tasks to others. This can result in an excessive workload and potential burnout. Encouraging him to delegate tasks appropriately will not only alleviate his own workload but also foster a more balanced and productive team environment.
'''

In [18]:
results = seq_chain(employee_review)



> Entering new SequentialChain chain...

> Finished chain.


In [19]:
print(results['review'])


Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexi

In [20]:
print(results['review_summary'])

Summary:

Joe Schmo, a software engineer, received a positive performance review. He is highly skilled and knowledgeable in programming languages, algorithms, and software development best practices. Joe collaborates well with cross-functional teams, actively seeking input and contributing valuable insights. He demonstrates initiative and self-motivation, taking the lead in seeking out new projects and making significant improvements in existing processes. Joe is adaptable, handling changing project requirements and learning new technologies seamlessly. His problem-solving skills are exceptional, approaching issues with a logical mindset and finding effective solutions. 

However, there are areas for improvement. Joe struggles with time management at times, resulting in missed deadlines or the need for additional support. Enhancing prioritization and time management techniques would increase efficiency. His written communication skills could also be improved to ensure clarity and avoid

In [21]:
print(results['weaknesses'])

Key employee weaknesses identified in this review summary:

1. Time management: Joe struggles with time management, leading to missed deadlines or the need for additional support. This suggests that he may not be effectively prioritizing tasks or managing his workload efficiently.

2. Written communication skills: Joe's written communication skills could be improved to ensure clarity and avoid confusion among team members. This weakness may indicate that he needs to work on effectively conveying his ideas and instructions in writing.

3. Delegation: Joe tends to take on too many responsibilities and hesitates to delegate tasks. This may result in an excessive workload and potential burnout. Encouraging appropriate delegation would help create a more balanced and productive team environment.


In [22]:
print(results['final_plan'])

Personalized Plan to Address and Fix the Identified Weaknesses:

1. Time Management:

a. Set Clear Goals and Priorities: Work with Joe to establish clear goals and priorities for each project or task. This will help him understand what needs to be done and prioritize his time accordingly.

b. Create a Schedule: Help Joe create a detailed schedule or to-do list that outlines specific tasks, deadlines, and estimated time required for completion. This will assist him in organizing his time effectively and ensure that he meets deadlines.

c. Time Blocking: Encourage Joe to allocate specific blocks of time for different tasks or activities. This will help him focus on one task at a time and avoid multitasking, which can lead to inefficiency.

d. Time Tracking: Have Joe track his time spent on different tasks for a week or two. This will provide insight into how he is currently allocating his time and identify areas where he can improve.

2. Written Communication Skills:

a. Provide Training

# LLMRouterChains

In [23]:
from langchain.chains import LLMRouterChain

```bash
           ------------------------------------
Input --> | Router | LLM Decides Chain | Chain | ---> Output
           ------------------------------------
```

### Router Templates

In [38]:
beginner_template = """You are a physics teachers who is really focused on beginners and explaining comples in simple to understand terms.
You assume no prior knowledge. Here is Question:\n{input}"""

In [39]:
expert_template = """You are a physics professor who explains physics topics to advanced audience members.
You can assume anyone you answer has a PhD in Physics. Here is a question:\n{input}"""

In [40]:
# Route Prompt Information
# [] Name, Description, Template

In [52]:
prompt_infos = [
    {
        'name': 'beginner physics',
        'description': 'Answers basic physics questions',
        'template': beginner_template,
    },
    {
        'name': 'advanced Physics',
        'description': 'Answers advanced physics questions',
        'template': expert_template,
    }
]

In [53]:
destination_chains = dict()
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=chat, prompt=prompt)
    destination_chains[name] = chain

In [54]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=chat,prompt=default_prompt)

In [55]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [56]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [57]:
destinations_str

'beginner physics: Answers basic physics questions\nadvanced Physics: Answers advanced physics questions'

In [58]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["question"],
    output_parser=RouterOutputParser(),
)

In [59]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
beginner physics: Answers basic physics questions
advanced Physics: Answers advanced physics question

In [60]:
router_chain = LLMRouterChain.from_llm(chat, router_prompt)

In [61]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [62]:
chain.run("How do magnets work?")



> Entering new MultiPromptChain chain...
beginner physics: {'input': 'How do magnets work?'}
> Finished chain.


'Great question! Let\'s dive into the fascinating world of magnets and understand how they work.\n\nMagnets are objects that produce a magnetic field, which is an invisible force field that surrounds them. This magnetic field is what allows magnets to attract or repel certain materials.\n\nNow, to understand how magnets work, we need to talk about tiny particles called electrons. Electrons are negatively charged particles that orbit around the nucleus of an atom. Every electron behaves like a tiny magnet, having a property called "spin."\n\nIn most materials, like wood or plastic, the electrons are oriented randomly, canceling out each other\'s magnetic effects. However, in certain materials like iron, nickel, and cobalt, the electrons tend to align in the same direction, creating tiny regions called magnetic domains.\n\nWhen you bring a magnet near an object made of one of these magnetic materials, the magnetic field of the magnet interacts with the magnetic domains in the material. I

In [63]:
chain.run("what is zeno effect?")



> Entering new MultiPromptChain chain...
beginner physics: {'input': 'what is the Zeno effect?'}
> Finished chain.


'The Zeno effect, also known as the quantum Zeno effect, is a fascinating concept in quantum mechanics. To understand it, let\'s first talk about a basic principle in quantum mechanics called the wave-particle duality.\n\nIn the quantum world, particles such as electrons or photons can behave both as particles and as waves. This means they can exist in multiple states simultaneously, which we call a superposition. However, when we observe or measure a particle, it "collapses" into a single state or position.\n\nNow, let\'s dive into the Zeno effect. Named after the ancient Greek philosopher Zeno of Elea, who posed paradoxes involving motion, the Zeno effect is a phenomenon where repeated measurements or observations can prevent a quantum system from evolving or changing its state.\n\nImagine you have a quantum system, like an atom, that is initially in a superposition of two states, let\'s say state A and state B. According to the laws of quantum mechanics, this system would naturally 

In [64]:
chain.run("How do Feynman Diagrams work?")



> Entering new MultiPromptChain chain...
advanced Physics: {'input': 'How do Feynman Diagrams work?'}
> Finished chain.


"Ah, Feynman diagrams, an elegant and powerful tool in the realm of quantum field theory. To understand how they work, let's dive into the fascinating world of particle interactions.\n\nFirst introduced by the renowned physicist Richard Feynman, Feynman diagrams provide a pictorial representation of particle interactions, specifically within the framework of quantum electrodynamics (QED) and quantum chromodynamics (QCD). They help us calculate the probabilities of various particle processes occurring.\n\nAt the heart of Feynman diagrams lies the concept of virtual particles. In quantum field theory, empty space is not really empty but is teeming with virtual particles spontaneously appearing and disappearing due to quantum fluctuations. These virtual particles are responsible for mediating the forces between real particles.\n\nEach Feynman diagram represents a specific interaction or process involving particles. The diagram consists of lines representing the particles involved, along w